efsdf

In [13]:
import bpy
import urllib.request, os
from mathutils import Matrix

# 1) Download your map PNG
def download_image(url, fname):
    dest = os.path.join(bpy.app.tempdir, fname)
    urllib.request.urlretrieve(url, dest)
    return dest

url = "https://raw.githubusercontent.com/Octoframes/bpy-workshop/refs/heads/main/Germany_location_map.png"
fname = "Germany_location_map.png"
image_path = download_image(url, fname)

# 2) Compute extents + center
north, south = 55.1, 47.2
west,  east  = 5.5, 15.5
width  = east  - west    # 10.0
height = north - south   #  7.9
center_x = (east + west) / 2.0   # 10.5
center_y = (north + south) / 2.0 # 51.15

# 3) Add a unit plane at the origin
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.mesh.primitive_plane_add(size=1, location=(0, 0, 0))
obj = bpy.context.active_object
obj.name = "MapPlane"
obj.select_set(True)
bpy.context.view_layer.objects.active = obj

# 4) Recenter its origin to its own geometry
bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='BOUNDS')

# 5) Build and apply a 4×4 scale matrix on the mesh data
m_x = Matrix.Scale(width,  4, (1, 0, 0))
m_y = Matrix.Scale(height, 4, (0, 1, 0))
obj.data.transform(m_x @ m_y)

# 6) Move the whole object to its geographic center
obj.location = (center_x, center_y, 0)

# 7) Create + assign the image‑texture material
mat = bpy.data.materials.new(name="MapMaterial")
mat.use_nodes = True
nodes = mat.node_tree.nodes
links = mat.node_tree.links

bsdf = nodes.get("Principled BSDF") or nodes.new("ShaderNodeBsdfPrincipled")
tex  = nodes.new("ShaderNodeTexImage")
# Load and pack the image into the .blend
img = bpy.data.images.load(image_path)
img.pack()
tex.image = img

# Link texture to BSDF
links.new(tex.outputs["Color"], bsdf.inputs["Base Color"])

# Assign material
obj.data.materials.clear()
obj.data.materials.append(mat)

# 8) Optional: pack all external data (including the image)
bpy.ops.file.pack_all()

# 9) Sanity‑check
print("Plane is named:", obj.name)
print(f"Final mesh dims: {obj.dimensions.x:.3f} × {obj.dimensions.y:.3f}")
print("Image packed?", img.packed_file is not None)


Info: No new files have been packed
Plane is named: MapPlane
Final mesh dims: 10.000 × 7.900
Image packed? True


In [15]:
center_x, center_y

(10.5, 51.150000000000006)